In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import pandas as pd
from typing import Tuple


In [26]:
def connection_scan_latest_arrival(timetable: pd.DataFrame, footpaths: pd.DataFrame, source_stop_id: str, destination_stop_id: str, arrival_time: str) -> Tuple[dict, dict]:
        """Use the connection scan algorithm (SCA) to find the latest possible departure from the source stop such
        that the destination stop can be reached on time.

        Args:
            timetable (pd.DataFrame): A DataFrame containing the timetable of all trips.
            footpaths (pd.DataFrame): A DataFrame containing the footpaths between stops.
            source_stop_id (str): The ID of the source stop.
            destination_stop_id (str): The ID of the destination stop.
            arrival_time (str): The desired arrival time at the destination stop.

        Returns:
            S (dict): It maintains the latest possible departure from depart station such that destination stop can eventually be reached on time.
            T (dict): It maintains True or False based on whether the destination can be reached using the given trip.
        """

        total_seconds_init = pd.to_timedelta('00:00:00').total_seconds()
        total_seconds_arrival = pd.to_timedelta(arrival_time).total_seconds()

        timetable_sorted = timetable.sort_values(by='arr_time').reset_index(drop=True)
        stop_ids = set(list(timetable['dep_stop']) + list(timetable['arr_stop']) + list(footpaths['stop_id_a']))

        # T maintains True or False based on whether the destination can be reached using the given trip
        trip_ids = set(timetable['trip_id'])
        T = dict.fromkeys(trip_ids, False)
        
        # S maintains the latest possible departure from depart station such that destination stop can eventually be reached on time
        # Use a dict also as value to improve readability
        S = dict.fromkeys(stop_ids, {
            'transport': None,
            'start_time': total_seconds_init,
            'start_stop': None,
            'arrival_time': None,
            'arrival_stop': None
        })

        S[destination_stop_id] = {
            'transport': None,
            'start_time': total_seconds_arrival,
            'start_stop': destination_stop_id,
            'arrival_time': None,
            'arrival_stop': None
        }
        
        # catch case where source == destination so loop quits early
        if source_stop_id == destination_stop_id:
            S[source_stop_id] = {
                'transport': None,
                'start_time': total_seconds_arrival,
                'start_stop': source_stop_id,
                'arrival_time': total_seconds_arrival,
                'arrival_stop': destination_stop_id
            }
            return S, T
            
        # init S entries for footpaths leading to destination stop
        for _, fp in footpaths[footpaths['stop_id_a'] == destination_stop_id].iterrows():
            S[fp['stop_id_b']] = {
                'transport': 'walking',
                'start_time': total_seconds_arrival - fp['duration'],
                'start_stop': fp['stop_id_b'],
                'arrival_time': total_seconds_arrival,
                'arrival_stop': destination_stop_id
            }

        # c_0 is the last connection in the timetable that arrives at its stop before the user's intended arrival time
        c_0 = timetable_sorted[timetable_sorted['arr_time'] <= total_seconds_arrival].iloc[-1]['connection_id'] if np.any(timetable_sorted['arr_time'] <= total_seconds_arrival) else None
        
        # timetable subset: from earliest arrival to c_0 arrival, in reverse order
        ## rss = reverse-sorted subset
        timetable_rss = timetable_sorted.iloc[:timetable_sorted.index[timetable_sorted['connection_id'] == c_0][0] + 1].iloc[::-1].reset_index(drop=True)

        i = 0

        # starting with the last possible connection and working earlier and earlier...
        ## in comments, current connection = connection currently being considered by loop
        for _, row in timetable_rss.iterrows():
            # if arrival time of current connection is lower than S[source stop], algorithm is completed
            ## this is because S[source stop] maintains the latest possible departure from source stop s.t. the destination can be reached
            ## if this condition is met, we've started considering connections that no longer need to be considered
            if S[source_stop_id]['start_time'] >= row['arr_time']:
                break
            # if current connection's departure time is later than all previously scanned connections departing from the same stop 
            # AND
            # if (we know the trip can be used to eventually reach the destination) OR (current connection's arrival time is earlier than departure time for all previously scanned connections leaving from the arrival stop)
            if (T[row['trip_id']] or S[row['arr_stop']]['start_time'] >= row['arr_time']) and (S[row['dep_stop']]['start_time'] < row['dep_time']):
                # indicate that the trip can be used to reach the destination (ensures later connections in same trip are not neglected)
                T[row['trip_id']] = True
                # log the current connection as the new latest-departure connection for the current departure stop
                S[row['dep_stop']] = {
                    'transport': row['trip_id'],
                    'start_time': row['dep_time'],
                    'start_stop': row['dep_stop'],
                    'arrival_time': row['arr_time'],
                    'arrival_stop': row['arr_stop']
                }
                # update latest-departure walking connection for stations close enough to current departure stop to reflect new latest departure from current departure stop 
                for _, fp in footpaths[footpaths['stop_id_a'] == row['dep_stop']].iterrows():
                    if S[fp['stop_id_b']]['start_time'] < ( row['dep_time'] - fp['duration']):
                        S[fp['stop_id_b']] = {
                            'transport': 'walking',
                            'start_time': row['dep_time'] - fp['duration'],
                            'start_stop': fp['stop_id_b'],
                            'arrival_time': row['dep_time'],
                            'arrival_stop': fp['stop_id_a']
                        }
            i += 1
            if i % 1000 == 0:
                try:
                    print(S['8501121:0:2'], stops_info[stops_info['stop_id'] == '8501121:0:2']['stop_name'].values[0])
                except:
                    pass
        return S, T

In [27]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from matplotlib import pyplot as plt

timetable = pd.read_csv('data/timetable.csv')
footpaths = pd.read_csv('data/footpaths.csv')
stops_info = pd.read_csv('data/stops.csv')
stops_info['stop_name_id'] = stops_info['stop_name'] + " (" + stops_info['stop_id'] + ")"

S, T = connection_scan_latest_arrival(timetable, footpaths, '8501215', '8501121:0:2', '20:00:00')

{'transport': None, 'start_time': 0.0, 'start_stop': None, 'arrival_time': None, 'arrival_stop': None} Renens VD
{'transport': None, 'start_time': 0.0, 'start_stop': None, 'arrival_time': None, 'arrival_stop': None} Renens VD
{'transport': None, 'start_time': 0.0, 'start_stop': None, 'arrival_time': None, 'arrival_stop': None} Lausanne
{'transport': None, 'start_time': 0.0, 'start_stop': None, 'arrival_time': None, 'arrival_stop': None} Lausanne, Grancy
{'transport': None, 'start_time': 0.0, 'start_stop': None, 'arrival_time': None, 'arrival_stop': None} Renens VD
{'transport': None, 'start_time': 0.0, 'start_stop': None, 'arrival_time': None, 'arrival_stop': None} Renens VD
{'transport': None, 'start_time': 0.0, 'start_stop': None, 'arrival_time': None, 'arrival_stop': None} Lausanne
{'transport': None, 'start_time': 0.0, 'start_stop': None, 'arrival_time': None, 'arrival_stop': None} Lausanne, Grancy
{'transport': None, 'start_time': 0.0, 'start_stop': None, 'arrival_time': None, 'ar